# Alguns tópicos adicionais sobre classes

## 1. Classes como objetos

Uma classe, além de definir um novo tipo de dados, é também um objeto, criado quando se define a classe.

In [1]:
class A:
    def f(self):
        print('Hi')

In [ ]:
A

Como já explicamos, se usamos esse objeto como se fosse uma função, um objeto da classe é criado e o método `__init__` é chamado para ele.

In [ ]:
a = A()

Todos os objetos tem um tipo, que é a classe usada para sua criação.

In [ ]:
type(a)

Note que a classe inclui o nome do módulo onde ela foi declarada (neste caso, o módulo corrente `__main__`).

Como a classe também é um objeto, ela também tem um tipo:

In [2]:
type(A)

type

Por sua vez, o tipo de `type` é o próprio `type`:

In [ ]:
type(type)

Isso vale para outros tipos:

In [3]:
type(int), type(float), type(str)

(type, type, type)

## 2. Atributos de classe

Como classes são objetos, podemos criar atributos nela. Isso é o que ocorre quando definimos um método, que é um atributo da classe que guarda uma função. Mas também podemos colocar variáveis simples.

In [4]:
class B:
    def f(self):
        print('Hello')
    x = 1

In [5]:
B.x

1

In [6]:
B.f

<function __main__.B.f(self)>

In [7]:
B.x = 2

In [8]:
B.x

2

Os atributos definidos na classe são também disponíveis nos objetos dessa classe (da mesma forma que os atributos que são métodos).

In [9]:
b = B()

In [10]:
b.f()

Hello


In [11]:
b.x

2

In [12]:
B.x += 2
b.x

4

Atributos da classe, acessados através de seus objetos, **têm o mesmo valor para todos os objetos da classe**, ao contrário de atributos do objeto (que são aquelos associados com `self` nos métodos da classe.

In [41]:
class C:
    x = 1
    def __init__(self):
        self.y = 2
    def f(self):
        print(f'Hello, I have {self.y}')

In [17]:
C.x

1

O atributo `y` é associado a objetos da classe, e portanto não é acessível pela classe, mas apenas aos objetos.

In [18]:
C.y

AttributeError: type object 'C' has no attribute 'y'

In [19]:
c = C()

In [20]:
c.y

2

Já o atributo `x`, além de ser acessado pela classe, pode ser acessados pelos objetos:

In [21]:
c.x

1

In [22]:
c2 = C()

In [23]:
c2.x

1

In [24]:
c2.y

2

In [25]:
c2.y = 3

In [27]:
c.y

2

In [28]:
C.x = 5

In [29]:
c.x

5

In [30]:
c2.x

5

Mas preste atenção pois as variáveis da classe são acessadas pelos objetos da mesma forma que variáeis globais são acessadas pelas funções, isto é, **os valores podem ser lidos a qualquer momento, mas se escrevemos no valor através do objeto, então criamos uma nova variável local do objeto**:

In [42]:
c3 = C()  # crio um objeto do tipo classe C e atribuo a ref dele à var c3
c3.x  # acessando a variável do escopo da classe 

1

In [43]:
C.x += 1
c3.x

2

In [44]:
c3.x = 10
c3.x, C.x

(10, 2)

In [37]:
c2.x

6

## 3. Escopo de objeto

A explicação desse comportamento está relacionada com o conceito de escopo. Cada objeto em Python **tem o seu próprio escopo**. Isso significa que podemos acrescentar atributos a qualquer momento em um objeto já existente. Esse novo atributo estará presente apenas nesse objeto, e não em outros objetos da classe.

In [ ]:
c2.z = 4

In [ ]:
c2.z

In [ ]:
c.z

Por outro lado, como a classe é também um objeto, podemos acrescentar novos atributos à classe, e eles serão visíveis a todos os objetos da classe.

In [ ]:
C.t = 7

In [ ]:
c.t

In [ ]:
c2.t

É possível também alterar a função associada com um método, apesar de isso **não ser uma prática recomendada**.

In [ ]:
c.f()

In [ ]:
C.f = lambda self: print('Bye!')

In [ ]:
c.f()

In [ ]:
c2.f()

Quando o Python tenta acessar um atributo em um objeto, ele **busca primeiro no escopo do objeto** e vai procurar no escopo da classe (e depois de suas classes base) se não for encontrado. Isso significa que **um atributo do objeto com mesmo nome do atributo da classe vai esconder o atributo da classe** quando acessado através do objeto (mas não ao acessar através da classe).

In [ ]:
class D:
    x = 1
    def __init__(self):
        self.x = 2
    def get_x(self):
        print (self.x)

In [ ]:
d = D()

In [ ]:
d.get_x()

In [ ]:
d.x

In [ ]:
D.x

Objetos de classes derivada também têm acesso aos atributos da classe base.

In [ ]:
class E(C):
    z = 2
    def g(self):
        print('Here')

In [ ]:
e = E()

In [ ]:
e.z

In [ ]:
e.x

Repare que esse processo de criação de atributos em objetos é exatamento o usado em código normais Python. Por exemplo, considere a classe abaixo:

In [ ]:
class X:
    def __init__(self, a):
        self._x = a
    def get(self):
        return self._x

Quando criamos um objeto dessa classe:

In [ ]:
x = X(5)

sabemos que o que acontece é que o Python cria o objeto e então chama o `__init__` da classe X para iniciar esse objeto. Quando esse método é chamado, `self` é uma referência para o objeto criado, e portanto `self._x = a` irá criar um novo atributo chamado `_x` nesse objeto, com o valor especificado pelo parâmetro de inicialização!

O que acontece é que, como `__init__` é garantidamente chamado para todos os objetos da classe, então garantimos que esse atributo realmente existirá em todos os objetos da classe.

Nos exemplos acima, como estamos criando separadamente atributos para objetos específicos, então não temos garantia de que o atributo existirá em todos os objetos da classe.

O mesmo vale para criação de novos atributos do objeto em outros métodos da classe (que não o `__init__`): é possíbel fazer, mas é melhor evitar pois, ao contrário do `__init__`, não podemos garantir que o método que cria o atributo tenha sido chamado antes de querermos usar o atributo. Por isso temos a regra de **sempre criar todos os atributos do objeto no método `__init__`**. Se você ainda não tiver valores para colocar nesse atributo, use `None`.

### 3.1. Exemplo

No código abaixo, usamos um atributo de classe para contar o número de objetos dessa classe que já foram criados.

Inicializamos o atributo em 0, e depois, como cada novo objeto criado irá executar o método `__init__`, então basta incrementar esse valor cada vez que o `__init__` for executado.

In [ ]:
class AutoCount:
    count = 0
    def __init__(self):
        AutoCount.count += 1

In [ ]:
x1 = AutoCount()

In [ ]:
AutoCount.count

In [ ]:
many = [AutoCount() for i in range(10)]

In [ ]:
AutoCount.count

In [ ]:
x1.count

## 4. Atributos em um dicionário

O atributo `__dict__` pode ser consultado para verificar os atributos de um objeto. Ele é um dicionário com a chave sendo o nome do atributo e o valor sendo o valor do atributo.

In [ ]:
type(a)

In [ ]:
a.__dict__

In [ ]:
A.__dict__

In [ ]:
c.__dict__

In [ ]:
C.__dict__

Note como os atributos associados ao objeto estão no `__dict__` do objeto, enquanto aqueles associados com a classe (incluindo os métodos), estão no `__dict__` da classe.

Da mesma forma que podemos criar novos atributos, podemos também apagar atributos existentes, usando o comando `del`.

In [ ]:
del C.t

In [ ]:
C.__dict__

In [ ]:
C.t

Uma outra forma de acessar os atributos associados a um objeto (mas sem ver seus valores) é o uso da função `dir`:

In [ ]:
dir(a)

In [ ]:
dir(c)

In [ ]:
dir(A)

In [ ]:
dir(C)

Note a grande quantidade de membros "mágicos" pré-definidos!

# Exercícios

  Qual a saída produzida nos códigos abaixo?

1. 
```python
class A:
    x = 1
    def __init__(self):
        self.y = 2
    def f(self):
        return self.x + self.y

a = A()
b = A()
b.y = 10
print(a.f(), b.f())
```

2. 
```python
class A:
    x = 1
    def __init__(self):
        self.y = 2
    def f(self):
        return self.x + self.y

a = A()
b = A()
A.x = 10
print(a.f(), b.f())
```

3. 
```python
class A:
    x = 1
    def __init__(self):
        self.y = 2
    def f(self):
        return self.x + self.y

a = A()
b = A()
b.x = 10
print(a.f(), b.f())
```

4.
```python
class A:
    x = 1
    def f(self):
        return self.x

class B(A):
    y = 10
    def __init__(self):
        self.z = 100
    def f(self):
        return self.x + self.y + self.z

a = A()
b = B()
c = B()
A.x = 2
b.y = 20
c.z = 200
print(a.f(), b.f(), c.f())
```